In [2]:
import pandas as pd
from haversine import haversine as hvs, Unit
import numpy as np

# corregir puntos de stops

In [37]:
def splitPoint(row):
    row['latitude'] = row['WKT'].split()[2].strip()[:-1]
    row['longitude'] = row['WKT'].split()[1].strip()[1:]
    return row

data_busstops = pd.read_csv('stop_points.csv')
# data_busstops = data_busstops.apply(splitPoint, axis=1)
data_busstops = data_busstops.sort_values(by='number').reset_index(drop=True)
data_busstops = data_busstops.drop(['number', 'WKT'], axis=1) # escoger los paraderos de ida
# data_busstops['id'] = range(36, len(data_busstops)+36)
data_busstops['id'] = data_busstops['id'].apply(lambda id: str(int(id)))
data_busstops['navigation'] = data_busstops['navigation'].values.astype(np.int64)
data_busstops['radio'] = data_busstops['radio'].values.astype(np.int64)

# data_busstops.append(pd.Series([0, 'start', 0, ]))
data_busstops

,id,name,navigation,latitude,longitude,radio
0,0,inicio,159,-13.554978,-71.856025,30
1,36,primer stop,304,-13.554812,-71.857300,18
2,37,segundo stop,281,-13.553294,-71.861944,13
3,38,Ccollana,283,-13.552296,-71.867299,9
4,39,Puente Huaccoto,283,-13.551592,-71.870211,10
...,...,...,...,...,...,...
79,114,Segundo,221,-13.547528,-71.985248,10
80,115,Reservorio,225,-13.549660,-71.988003,10
81,116,Cuarto,230,-13.550408,-71.988880,10
82,117,Quinto,203,-13.550841,-71.989448,10


In [39]:
# df_stops_ida = pd.read_csv('busstops_ida.csv')
# data_busstops['radio'] = df_stops_ida['radio']
# data_busstops

In [40]:
data_busstops.to_csv('busstops_ida.csv')

# Extraer en CSV

In [7]:
dt['vehicle_id_id'].unique()[13]

'SJ35'

In [8]:
# leer dataset monitoring
dt = pd.read_csv('monitoring.csv')

# parametros de extraccion
lap = 5
vehicle =  dt['vehicle_id_id'].unique()[13]

# diseño del df
dt_veh = dt[dt['vehicle_id_id'] == vehicle].copy()
dt_veh['date'] += ' ' + dt_veh['time']
dt_veh.drop('time', axis=1)
dt_veh = dt_veh.rename(columns={'date':'date_time'}, inplace=False)
dt_veh = dt_veh[dt_veh['lap'] == lap]

# ordenar por data time
dt_veh = dt_veh.sort_values(by='date_time').reset_index(drop=True)
dt_veh

,id,date_time,time,lap,latitude,longitude,velocity,navigation,vehicle_id_id
0,22065540,2022-07-14 15:29:17,15:29:17,5,-13.527892,-71.983110,13,18,SJ35
1,22065599,2022-07-14 15:29:33,15:29:33,5,-13.527807,-71.982356,27,82,SJ35
2,22065734,2022-07-14 15:30:14,15:30:14,5,-13.527752,-71.981618,11,28,SJ35
3,22065804,2022-07-14 15:30:34,15:30:34,5,-13.527188,-71.981937,19,290,SJ35
4,22065845,2022-07-14 15:30:46,15:30:46,5,-13.526988,-71.982601,21,290,SJ35
...,...,...,...,...,...,...,...,...,...
157,22078741,2022-07-14 16:27:35,16:27:35,5,-13.554582,-71.857696,45,123,SJ35
158,22078867,2022-07-14 16:28:11,16:28:11,5,-13.555058,-71.856966,25,122,SJ35
159,22078926,2022-07-14 16:28:23,16:28:23,5,-13.555595,-71.856224,24,126,SJ35
160,22079011,2022-07-14 16:28:46,16:28:46,5,-13.555559,-71.855643,8,0,SJ35


In [9]:
if 'lap' not in dt_veh.columns:
    dt_veh.to_csv('monitoring_veh/' + str(vehicle) + '.csv')
else:
    dt_veh.to_csv('monitoring_veh_lap/' + str(vehicle) + '-lap' + str(lap) + '.csv')